# Transit Delay Prediction - Initial Analysis

**Project:** Vancouver Transit Delay Prediction Tool  
**Threshold:** Major delays defined as ≥10 minutes  
**Data Source:** TransLink GTFS Realtime API

---

## Table of Contents
1. [Question 6: Exploratory Data Analysis](#question-6)
2. [Question 7: Baseline Model (Without ML)](#question-7)
3. [Question 8: Logistic Regression Model](#question-8)
4. [Summary and Conclusions](#summary)

## Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

: 

## Load Data

In [ ]:
import io
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config

BUCKET = "bus-delay-forecast"
PREFIX = "processed/gtfs_rt/trip_updates_parquet/split=test/"  # correct parquet prefix

# Anonymous / unsigned S3 client
s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

def list_parquet_keys(bucket: str, prefix: str) -> list[str]:
    keys = []
    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            k = obj["Key"]
            if k.endswith(".parquet"):
                keys.append(k)
    return sorted(keys)

keys = list_parquet_keys(BUCKET, PREFIX)
print("Parquet files found:", len(keys))
if not keys:
    raise FileNotFoundError(f"No parquet files found at s3://{BUCKET}/{PREFIX}")

dfs = []
for i, key in enumerate(keys, 1):
    body = s3.get_object(Bucket=BUCKET, Key=key)["Body"].read()
    dfs.append(pd.read_parquet(io.BytesIO(body), engine="pyarrow"))
    if i % 10 == 0 or i == len(keys):
        print(f"Loaded {i}/{len(keys)} files")

data = pd.concat(dfs, ignore_index=True)

# --- Same feature engineering as before ---
data["timestamp"] = pd.to_datetime(data["feed_timestamp"], unit="s", utc=True)
data["timestamp_pt"] = data["timestamp"].dt.tz_convert("America/Vancouver")

data["hour"] = data["timestamp_pt"].dt.hour
data["day_of_week"] = data["timestamp_pt"].dt.dayofweek
data["day_name"] = data["timestamp_pt"].dt.day_name()
data["is_weekend"] = (data["day_of_week"] >= 5).astype(int)
data["is_rush_hour"] = data["hour"].isin([7, 8, 9, 16, 17, 18]).astype(int)

if "delay_min" not in data.columns and "delay_sec" in data.columns:
    data["delay_min"] = data["delay_sec"] / 60.0

data["major_delay"] = data["delay_min"] >= 10
data["delay_10plus"] = data["major_delay"].astype("int8")

print(f"\nTotal records: {len(data):,}")
print(f"Major delays (≥10 min): {int(data['major_delay'].sum()):,} "
      f"({data['major_delay'].mean()*100:.2f}%)")

data.head()


In [ ]:
# -----------------------------
# Summary stats you asked for
# -----------------------------
total_rows = len(data)
major = int(data["major_delay"].sum())
non_major = total_rows - major

unique_trips = data["trip_id"].nunique() if "trip_id" in data.columns else None
unique_stops = data["stop_id"].nunique() if "stop_id" in data.columns else None

print(f"\nTotal raw rows: {total_rows:,}")

print(f"Unique trips (trip_id): {unique_trips:,}" if unique_trips is not None else "trip_id column not found")
print(f"Unique stops (stop_id): {unique_stops:,}" if unique_stops is not None else "stop_id column not found")

print(f"\nMajor delay (>=10 min): {major:,} ({major/total_rows*100:.2f}%)")
print(f"No major delay (<10 min): {non_major:,} ({non_major/total_rows*100:.2f}%)")
print(f"Class imbalance ratio (non_major : major) = {non_major / max(major, 1):.1f}:1")

---
<a id="question-6"></a>
# Question 6: Exploratory Data Analysis

**Goal:** Plot distributions of features and outcomes to understand the dataset characteristics.

## 6.1 Delay Distribution

In [ ]:
# Plot delay distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Full distribution
axes[0].hist(data['delay_min'], bins=100, range=(-30, 60), edgecolor='black', alpha=0.7, color='#3498db')
axes[0].axvline(x=10, color='red', linestyle='--', linewidth=2, label='Major Delay Threshold (10 min)')
axes[0].axvline(x=0, color='green', linestyle='--', linewidth=1, alpha=0.5, label='On Time')
axes[0].set_xlabel('Delay (minutes)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Transit Delays', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Outcome variable
outcome_counts = data['major_delay'].value_counts()
colors = ['#2ecc71', '#e74c3c']
bars = axes[1].bar(['No Major Delay\n(< 10 min)', 'Major Delay\n(≥ 10 min)'], 
                   outcome_counts.values, color=colors, edgecolor='black', linewidth=1.5)
total = len(data)
for i, bar in enumerate(bars):
    height = bar.get_height()
    percentage = (height / total) * 100
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height):,}\n({percentage:.1f}%)',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Observations', fontsize=12)
axes[1].set_title('Outcome Variable Distribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Mean delay: {data['delay_min'].mean():.2f} minutes")
print(f"Median delay: {data['delay_min'].median():.2f} minutes")
print(f"Max delay: {data['delay_min'].max():.2f} minutes")
print(f"Min delay: {data['delay_min'].min():.2f} minutes")

## 6.2 Temporal Patterns

In [ ]:
# Temporal analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Delays by hour (box plot)
sns.boxplot(data=data, x='hour', y='delay_min', ax=axes[0, 0], color='#3498db')
axes[0, 0].axhline(y=10, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Threshold')
axes[0, 0].set_xlabel('Hour of Day')
axes[0, 0].set_ylabel('Delay (minutes)')
axes[0, 0].set_title('Delay Distribution by Hour of Day')
axes[0, 0].legend()
axes[0, 0].set_ylim(-30, 60)

# Delay rate by hour
hourly_delay_rate = data.groupby('hour')['major_delay'].mean() * 100
axes[0, 1].bar(hourly_delay_rate.index, hourly_delay_rate.values, color='#e74c3c', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Hour of Day')
axes[0, 1].set_ylabel('Major Delay Rate (%)')
axes[0, 1].set_title('Percentage of Trips with Major Delays by Hour')
axes[0, 1].set_xticks(range(24))
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Day of week analysis
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = data.groupby('day_name').size().reindex(day_order)
axes[1, 0].bar(range(len(day_counts)), day_counts.values, color='#3498db', edgecolor='black', alpha=0.7)
axes[1, 0].set_xticks(range(len(day_order)))
axes[1, 0].set_xticklabels(day_order, rotation=45, ha='right')
axes[1, 0].set_ylabel('Number of Observations')
axes[1, 0].set_title('Observations by Day of Week')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Delay rate by day
day_delay_rate = data.groupby('day_name')['major_delay'].mean().reindex(day_order) * 100
axes[1, 1].bar(range(len(day_delay_rate)), day_delay_rate.values, color='#e74c3c', edgecolor='black', alpha=0.7)
axes[1, 1].set_xticks(range(len(day_order)))
axes[1, 1].set_xticklabels(day_order, rotation=45, ha='right')
axes[1, 1].set_ylabel('Major Delay Rate (%)')
axes[1, 1].set_title('Major Delay Rate by Day of Week')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Convert for canva
# Delay rate by hour (already computed)
hourly_delay_rate = data.groupby('hour')['major_delay'].mean() * 100

# Convert to DataFrame
hourly_delay_df = hourly_delay_rate.reset_index()
hourly_delay_df.columns = ['Hour of Day', 'Major Delay Rate (%)']

hourly_delay_df.to_excel(
    "hourly_major_delay_rate.xlsx",
    index=False
)


## 6.3 Route Analysis

In [ ]:
# Route analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top routes by volume
top_routes = data['route_id'].value_counts().head(15)
axes[0].barh(range(len(top_routes)), top_routes.values, color='#3498db', edgecolor='black')
axes[0].set_yticks(range(len(top_routes)))
axes[0].set_yticklabels([f"Route {int(r)}" for r in top_routes.index])
axes[0].set_xlabel('Number of Observations')
axes[0].set_title('Top 15 Routes by Volume')
axes[0].grid(True, alpha=0.3, axis='x')

# Delay rate by top routes
top_route_ids = data['route_id'].value_counts().head(15).index
top_route_data = data[data['route_id'].isin(top_route_ids)]
route_delay_rate = top_route_data.groupby('route_id')['major_delay'].mean() * 100
route_delay_rate = route_delay_rate.sort_values(ascending=True)
colors_route = ['#e74c3c' if r > 10 else '#3498db' for r in route_delay_rate.values]
axes[1].barh(range(len(route_delay_rate)), route_delay_rate.values, color=colors_route, edgecolor='black')
axes[1].set_yticks(range(len(route_delay_rate)))
axes[1].set_yticklabels([f"Route {int(r)}" for r in route_delay_rate.index])
axes[1].set_xlabel('Major Delay Rate (%)')
axes[1].set_title('Major Delay Rate by Top 15 Routes')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Question 6: Key Findings

**Summary:**
- Most buses run on time (median delay: -0.18 minutes)
- Major delays (≥10 min) represent 8.05% of trips
- Class imbalance ratio: 11.4:1
- Temporal and route-level variation exists
- Some routes consistently have higher delay rates

---
<a id="question-7"></a>
# Question 7: Baseline Model (Without ML)

**Goal:** Create simple baseline predictors to establish performance benchmarks.

In [ ]:
# Hour bucket (use Vancouver time)
data["ts_hour"] = data["timestamp_pt"].dt.floor("H")

baseline_tbl = (
    data.groupby(["ts_hour", "route_id"], as_index=False)
        .agg(
            delay_10plus=("delay_10plus", "max"),   # any major delay in that hour+route
            is_rush_hour=("is_rush_hour", "max")    # hour-level rush flag
        )
)

# --- Time-based split on the compact table ---
baseline_tbl = baseline_tbl.sort_values("ts_hour").reset_index(drop=True)

split_idx = int(len(baseline_tbl) * 0.8)
train_data = baseline_tbl.iloc[:split_idx]
test_data  = baseline_tbl.iloc[split_idx:]

# Target
y_true = test_data["delay_10plus"].astype("int8").to_numpy()

print(f"Baseline table rows: {len(baseline_tbl):,}")
print(f"Train: {len(train_data):,} | Test: {len(test_data):,}")
print(f"Test delay rate: {test_data['delay_10plus'].mean()*100:.2f}%")

In [ ]:
test_data.head()

## 7.1 Baseline 1: Majority Class

In [ ]:
# Baseline 1: Always predict "No Delay"
baseline1_pred = np.zeros(len(test_data), dtype=bool)

baseline1_accuracy = accuracy_score(y_true, baseline1_pred)
baseline1_precision = precision_score(y_true, baseline1_pred, zero_division=0)
baseline1_recall = recall_score(y_true, baseline1_pred, zero_division=0)
baseline1_f1 = f1_score(y_true, baseline1_pred, zero_division=0)

print("Baseline 1: Majority Class (Always predict 'No Delay')")
print(f"  Accuracy:  {baseline1_accuracy:.4f}")
print(f"  Precision: {baseline1_precision:.4f}")
print(f"  Recall:    {baseline1_recall:.4f}")
print(f"  F1 Score:  {baseline1_f1:.4f}")
print("\n⚠️  Problem: 0% recall - never identifies actual delays!")

## 7.2 Baseline 2: Route-Based Predictor

In [ ]:
# Baseline 2: Route-based predictor
route_delay_rates = train_data.groupby('route_id')['delay_10plus'].mean()
threshold = 0.05
high_delay_routes = route_delay_rates[route_delay_rates > threshold].index

baseline2_pred = test_data['route_id'].isin(high_delay_routes).values

baseline2_accuracy = accuracy_score(y_true, baseline2_pred)
baseline2_precision = precision_score(y_true, baseline2_pred, zero_division=0)
baseline2_recall = recall_score(y_true, baseline2_pred, zero_division=0)
baseline2_f1 = f1_score(y_true, baseline2_pred, zero_division=0)

print(f"Baseline 2: Route-Based (Predict delay if route delay rate > {threshold*100:.0f}%)")
print(f"  High-delay routes identified: {len(high_delay_routes)}")
print(f"  Accuracy:  {baseline2_accuracy:.4f}")
print(f"  Precision: {baseline2_precision:.4f}")
print(f"  Recall:    {baseline2_recall:.4f}")
print(f"  F1 Score:  {baseline2_f1:.4f}")

## 7.3 Baseline 3: Time-Based Predictor

In [ ]:
# Baseline 3: Rush hour predictor
baseline3_pred = test_data['is_rush_hour'].values

baseline3_accuracy = accuracy_score(y_true, baseline3_pred)
baseline3_precision = precision_score(y_true, baseline3_pred, zero_division=0)
baseline3_recall = recall_score(y_true, baseline3_pred, zero_division=0)
baseline3_f1 = f1_score(y_true, baseline3_pred, zero_division=0)

print("Baseline 3: Time-Based (Predict delay during rush hours)")
print(f"  Accuracy:  {baseline3_accuracy:.4f}")
print(f"  Precision: {baseline3_precision:.4f}")
print(f"  Recall:    {baseline3_recall:.4f}")
print(f"  F1 Score:  {baseline3_f1:.4f}")

## 7.4 Baseline Comparison

In [ ]:
# Compare baselines
baseline_results = pd.DataFrame({
    'Baseline': ['Majority Class', 'Route-Based', 'Time-Based (Rush Hour)'],
    'Accuracy': [baseline1_accuracy, baseline2_accuracy, baseline3_accuracy],
    'Precision': [baseline1_precision, baseline2_precision, baseline3_precision],
    'Recall': [baseline1_recall, baseline2_recall, baseline3_recall],
    'F1 Score': [baseline1_f1, baseline2_f1, baseline3_f1]
})

print("\n" + "="*70)
print("BASELINE COMPARISON")
print("="*70)
print(baseline_results.to_string(index=False))

best_baseline = baseline_results.loc[baseline_results['F1 Score'].idxmax()]
print(f"\n🏆 Best Baseline: {best_baseline['Baseline']}")
print(f"   F1 Score: {best_baseline['F1 Score']:.4f}")

# Visualize comparison
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['#3498db', '#e74c3c', '#2ecc71']

for idx, metric in enumerate(metrics):
    axes[idx].bar(baseline_results['Baseline'], baseline_results[metric], color=colors, edgecolor='black', alpha=0.7)
    axes[idx].set_ylabel(metric)
    axes[idx].set_title(f'{metric} Comparison')
    axes[idx].set_ylim(0, 1)
    axes[idx].tick_params(axis='x', rotation=15)
    axes[idx].grid(True, alpha=0.3, axis='y')

plt.suptitle('Baseline Model Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Question 7: Key Findings

**Summary:**
- Majority class baseline: High accuracy (93%) but useless (0% recall)
- Route-based baseline: Best performer with F1 = 0.0871
- Time-based baseline: No predictive value in current dataset
- **Benchmark to beat:** F1 = 0.0871

## Prepare Features and Split Data for regression models

In [ ]:
cutoff_ts = train_data["ts_hour"].max()
train_df = data[data["timestamp_pt"] < cutoff_ts]
test_df  = data[data["timestamp_pt"] >= cutoff_ts]


## Feature Engineering

In [ ]:


# Route-level stats
route_stats = train_df.groupby('route_id').agg({
    'delay_min': ['mean', 'std', 'count'],
    'delay_10plus': 'mean'
}).reset_index()
route_stats.columns = ['route_id', 'route_avg_delay', 'route_std_delay', 'route_trip_count', 'route_delay_rate']

# Stop-level stats
stop_stats = train_df.groupby('stop_id').agg({
    'delay_min': ['mean', 'count'],
    'delay_10plus': 'mean'
}).reset_index()
stop_stats.columns = ['stop_id', 'stop_avg_delay', 'stop_trip_count', 'stop_delay_rate']

# Merge stats into BOTH train/test
train_df = train_df.merge(route_stats, on='route_id', how='left').merge(stop_stats, on='stop_id', how='left')
test_df  = test_df.merge(route_stats, on='route_id', how='left').merge(stop_stats, on='stop_id', how='left')

# Global fallback values (for unseen route/stop in test)
global_avg_delay  = train_df['delay_min'].mean()
global_std_delay  = train_df['delay_min'].std()
global_delay_rate = train_df['delay_10plus'].mean()

# Fill NaNs
train_df['route_std_delay'] = train_df['route_std_delay'].fillna(0)

test_df['route_avg_delay']  = test_df['route_avg_delay'].fillna(global_avg_delay)
test_df['route_std_delay']  = test_df['route_std_delay'].fillna(global_std_delay if pd.notna(global_std_delay) else 0)
test_df['route_delay_rate'] = test_df['route_delay_rate'].fillna(global_delay_rate)

test_df['stop_avg_delay']   = test_df['stop_avg_delay'].fillna(global_avg_delay)
test_df['stop_delay_rate']  = test_df['stop_delay_rate'].fillna(global_delay_rate)




In [ ]:
data["trip_id"].nunique()


---
<a id="question-8"></a>
# Question 8: Logistic Regression Model

**Goal:** Build a logistic regression model to beat the baseline F1 score of 0.0871.

### Scale train & test for logistic

In [ ]:
from sklearn.preprocessing import StandardScaler

feature_cols = [
    'hour', 'day_of_week', 'is_weekend', 'is_rush_hour',
    'route_avg_delay', 'route_std_delay', 'route_delay_rate',
    'stop_avg_delay', 'stop_delay_rate', 'stop_sequence'
]

X_train = train_df[feature_cols].fillna(0)
X_test  = test_df[feature_cols].fillna(0)

y_train = train_df['delay_10plus'].values
y_test  = test_df['delay_10plus'].values

print(f"Training set: {len(X_train):,} samples ({y_train.sum()} delays, {y_train.mean()*100:.2f}%)")
print(f"Test set: {len(X_test):,} samples ({y_test.sum()} delays, {y_test.mean()*100:.2f}%)")

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("\n Features scaled (logistic-ready)")


## 8.1 Train Logistic Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [ ]:
# Train model with class weighting
logit_model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)

"""
logit_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logit", LogisticRegression(
        class_weight="balanced",
        C=0.1,
        max_iter=1000,
        solver="lbfgs",
        random_state=42,
        n_jobs=-1
    ))
])
"""

logit_model.fit(X_train_scaled, y_train)

# Make predictions (rename!)
y_pred_train_logit = logit_model.predict(X_train_scaled)
y_pred_test_logit  = logit_model.predict(X_test_scaled)

print(" Logistic model trained successfully!")

## 8.2 Evaluate Model Performance

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# =========================
# CALCULATE METRICS (LOGISTIC)
# =========================
logit_accuracy = accuracy_score(y_test, y_pred_test_logit)
logit_precision = precision_score(y_test, y_pred_test_logit, zero_division=0)
logit_recall = recall_score(y_test, y_pred_test_logit, zero_division=0)
logit_f1 = f1_score(y_test, y_pred_test_logit, zero_division=0)

print("="*70)
print("TEST SET PERFORMANCE (LOGISTIC REGRESSION)")
print("="*70)
print(f"Accuracy:  {logit_accuracy:.4f} ({logit_accuracy*100:.2f}%)")
print(f"Precision: {logit_precision:.4f}")
print(f"Recall:    {logit_recall:.4f}")
print(f"F1 Score:  {logit_f1:.4f}")

# =========================
# CONFUSION MATRIX
# =========================
cm = confusion_matrix(y_test, y_pred_test_logit)
print("\nConfusion Matrix (Logistic):")
print(f"  True Negatives:  {cm[0,0]:,}")
print(f"  False Positives: {cm[0,1]:,}")
print(f"  False Negatives: {cm[1,0]:,}")
print(f"  True Positives:  {cm[1,1]:,}")

# =========================
# COMPARE TO BASELINE
# =========================
baseline_f1 = baseline2_f1
improvement = ((logit_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0

print("\n" + "="*70)
print("COMPARISON TO BASELINE")
print("="*70)
print(f"Baseline F1 (Route-Based):     {baseline_f1:.4f}")
print(f"Logistic Regression F1:        {logit_f1:.4f}")
print(f"Improvement:                   {improvement:+.1f}%")

if logit_f1 > baseline_f1:
    print("\n SUCCESS: Logistic regression beats the baseline!")
else:
    print("\n Logistic regression did not beat baseline")


## 8.3 Feature Importance Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# =========================
# FEATURE IMPORTANCE (LOGISTIC REGRESSION)
# =========================
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    #'coefficient': logit_model.named_steps["logit"].coef_[0]
    'coefficient': logit_model.coef_[0]
})

feature_importance['abs_coefficient'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values('abs_coefficient', ascending=False)

print("="*70)
print("FEATURE IMPORTANCE (LOGISTIC REGRESSION)")
print("="*70)
print("\nTop 10 Most Important Features:")
for idx, row in feature_importance.head(10).iterrows():
    direction = "↑ Increases" if row['coefficient'] > 0 else "↓ Decreases"
    print(f"  {row['feature']:25s} {direction} delay risk (coef: {row['coefficient']:+.4f})")

# =========================
# VISUALIZE FEATURE IMPORTANCE
# =========================
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(10)
colors_feat = ['#e74c3c' if c > 0 else '#3498db' for c in top_features['coefficient']]

plt.barh(range(len(top_features)), top_features['coefficient'],
         color=colors_feat, edgecolor='black', alpha=0.7)

plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Coefficient Value', fontsize=12)
plt.title('Top 10 Feature Importance\n(Positive = Increases Delay Risk)',
          fontsize=14, fontweight='bold')

plt.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()


## 8.4 Model Comparison Visualization

In [ ]:
# =========================
# COMPARE BASELINE vs LOGISTIC
# =========================
model_comparison = pd.DataFrame({
    'Model': ['Baseline (Route-Based)', 'Logistic Regression'],
    'Accuracy': [baseline2_accuracy, logit_accuracy],
    'Precision': [baseline2_precision, logit_precision],
    'Recall': [baseline2_recall, logit_recall],
    'F1 Score': [baseline2_f1, logit_f1]
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['#3498db', '#e74c3c']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(
        model_comparison['Model'],
        model_comparison[metric],
        color=colors,
        edgecolor='black',
        alpha=0.7
    )
    ax.set_ylabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='x', rotation=15)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.,
            height,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold'
        )

plt.suptitle('Final Model Performance Comparison\n(Baseline vs Logistic Regression)',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + model_comparison.to_string(index=False))


---
<a id="question-9"></a>
# Question 9: XGBoost Regression Model

**Goal:** Build a XGBoost regression model to beat the losgistic regression F1 score of 0.462.

### Handle the imbalance in the data

In [ ]:
pos = y_train.sum()
neg = len(y_train) - pos
scale_pos_weight = neg / pos

print(f"scale_pos_weight = {scale_pos_weight:.2f}")

XGBoost internally treats each delayed trip as ~11× more important than an on-time trip during training.

## 9.1 Train & fit the model 

In [ ]:
from xgboost import XGBClassifier
# Train XGBoost model
xgb = XGBClassifier(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=6,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1
)


# Fit XGBoost model

xgb.fit(X_train, y_train)
print(" XGBoost model trained")

## 9.2 Predict

In [ ]:
# Probabilities
y_prob_test_xgb = xgb.predict_proba(X_test)[:, 1]

# Default threshold (0.5)
y_pred_test_xgb = (y_prob_test_xgb >= 0.5).astype(int)

## 9.3 Validate the model

### ROC AUC

In [ ]:
from sklearn.metrics import roc_auc_score
print("ROC AUC:", roc_auc_score(y_test, y_prob_test_xgb))

### Confusion matrix

In [ ]:
# =========================
# XGBOOST PREDICTIONS
# =========================
threshold = 0.5

y_prob_test_xgb = xgb.predict_proba(X_test)[:, 1]
y_pred_test_xgb = (y_prob_test_xgb >= threshold).astype(int)

# =========================
# CALCULATE METRICS (XGBOOST)
# =========================
xgb_accuracy = accuracy_score(y_test, y_pred_test_xgb)
xgb_precision = precision_score(y_test, y_pred_test_xgb, zero_division=0)
xgb_recall = recall_score(y_test, y_pred_test_xgb, zero_division=0)
xgb_f1 = f1_score(y_test, y_pred_test_xgb, zero_division=0)

print("="*70)
print("TEST SET PERFORMANCE (XGBOOST)")
print("="*70)
print(f"Accuracy:  {xgb_accuracy:.4f} ({xgb_accuracy*100:.2f}%)")
print(f"Precision: {xgb_precision:.4f}")
print(f"Recall:    {xgb_recall:.4f}")
print(f"F1 Score:  {xgb_f1:.4f}")

# =========================
# CONFUSION MATRIX
# =========================
cm_xgb = confusion_matrix(y_test, y_pred_test_xgb)
print("\nConfusion Matrix (XGBoost):")
print(f"  True Negatives:  {cm_xgb[0,0]:,}")
print(f"  False Positives: {cm_xgb[0,1]:,}")
print(f"  False Negatives: {cm_xgb[1,0]:,}")
print(f"  True Positives:  {cm_xgb[1,1]:,}")


### Compare to the baseline and the logistic regression

In [ ]:
# xgb_accuracy, xgb_precision, xgb_recall, xgb_f1

baseline_accuracy = baseline2_accuracy
baseline_precision = baseline2_precision
baseline_recall = baseline2_recall
baseline_f1 = baseline2_f1

# =========================
# % IMPROVEMENTS
# =========================
xgb_vs_base = ((xgb_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0
logit_vs_base = ((logit_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0
xgb_vs_logit = ((xgb_f1 - logit_f1) / logit_f1) * 100 if logit_f1 > 0 else 0

print("\n" + "="*70)
print("COMPARISON (BASELINE vs LOGISTIC vs XGBOOST)")
print("="*70)

print(f"Baseline F1 (Route-Based):     {baseline_f1:.4f}")
print(f"Logistic Regression F1:        {logit_f1:.4f}   ({logit_vs_base:+.1f}% vs baseline)")
print(f"XGBoost F1:                    {xgb_f1:.4f}   ({xgb_vs_base:+.1f}% vs baseline)")
print("-"*70)
print(f"XGBoost vs Logistic (F1):      {xgb_vs_logit:+.1f}%")

print("\nDetailed Metrics (Test Set):")
print(f"{'Model':<22} {'Acc':>8} {'Prec':>8} {'Recall':>8} {'F1':>8}")
print(f"{'-'*22} {'-'*8:>8} {'-'*8:>8} {'-'*8:>8} {'-'*8:>8}")
print(f"{'Baseline (Route)':<22} {baseline_accuracy:>8.4f} {baseline_precision:>8.4f} {baseline_recall:>8.4f} {baseline_f1:>8.4f}")
print(f"{'Logistic Regression':<22} {logit_accuracy:>8.4f} {logit_precision:>8.4f} {logit_recall:>8.4f} {logit_f1:>8.4f}")
print(f"{'XGBoost':<22} {xgb_accuracy:>8.4f} {xgb_precision:>8.4f} {xgb_recall:>8.4f} {xgb_f1:>8.4f}")

# =========================
# SIMPLE WINNER SUMMARY
# =========================
print("\n" + "="*70)
print("RESULT SUMMARY")
print("="*70)

best_model = max(
    [("Baseline (Route)", baseline_f1), ("Logistic Regression", logit_f1), ("XGBoost", xgb_f1)],
    key=lambda x: x[1]
)[0]

if best_model == "XGBoost" and xgb_f1 > baseline_f1 and xgb_f1 > logit_f1:
    print(" BEST: XGBoost is the top performer and beats both Logistic and Baseline.")
elif best_model == "Logistic Regression" and logit_f1 > baseline_f1 and logit_f1 > xgb_f1:
    print(" BEST: Logistic Regression is the top performer and beats both XGBoost and Baseline.")
elif best_model == "Baseline (Route)":
    print("  Baseline is still the strongest by F1. Consider threshold tuning / more features.")
elif xgb_f1 > baseline_f1:
    print(" XGBoost beats the Baseline, but does not beat Logistic.")
elif logit_f1 > baseline_f1:
    print(" Logistic beats the Baseline, but XGBoost does not.")
else:
    print(" Neither Logistic nor XGBoost beats the Baseline.")


### Compare XG Boost to the best model

In [ ]:
# =========================
# COMPARE LOGISTIC vs XGBOOST
# =========================
model_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'XGBoost'],
    'Accuracy': [logit_accuracy, xgb_accuracy],
    'Precision': [logit_precision, xgb_precision],
    'Recall': [logit_recall, xgb_recall],
    'F1 Score': [logit_f1, xgb_f1]
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['#e74c3c', "#3498db"]  # logistic, xgboost

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(
        model_comparison['Model'],
        model_comparison[metric],
        color=colors,
        edgecolor='black',
        alpha=0.7
    )
    ax.set_ylabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='x', rotation=15)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.,
            height,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold'
        )

plt.suptitle('Final Model Performance Comparison\n(Logistic Regression vs XGBoost)',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + model_comparison.to_string(index=False))

---
<a id="question-10"></a>
# Question 10: Random Forest Regression Model

**Goal:** Build a XGBoost regression model to beat the Losgistic regression F1 score of 0.462.

## 10.1 Train & fit the model

In [ ]:
# =========================
# TRAIN RANDOM FOREST
# =========================
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    class_weight='balanced_subsample',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

print(" Random Forest model trained")

## 10.2 Prediction

In [ ]:
threshold = 0.5

y_prob_test_rf = rf.predict_proba(X_test)[:, 1]
y_pred_test_rf = (y_prob_test_rf >= threshold).astype(int)

## 10.3 Validate the model

### ROC AUC

In [ ]:
rf_auc = roc_auc_score(y_test, y_prob_test_rf)
print(f"ROC AUC (Random Forest): {rf_auc:.4f}")

### Confussion Matrix

In [ ]:
# =========================
# RANDOM FOREST
# =========================
rf_accuracy = accuracy_score(y_test, y_pred_test_rf)
rf_precision = precision_score(y_test, y_pred_test_rf, zero_division=0)
rf_recall = recall_score(y_test, y_pred_test_rf, zero_division=0)
rf_f1 = f1_score(y_test, y_pred_test_rf, zero_division=0)

print("="*70)
print("TEST SET PERFORMANCE (RANDOM FOREST)")
print("="*70)
print(f"Accuracy:  {rf_accuracy:.4f} ({rf_accuracy*100:.2f}%)")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall:    {rf_recall:.4f}")
print(f"F1 Score:  {rf_f1:.4f}")

# =========================
# CONFUSION MATRIX
# =========================
cm_rf = confusion_matrix(y_test, y_pred_test_rf)
print("\nConfusion Matrix (Random Forest):")
print(f"  True Negatives:  {cm_rf[0,0]:,}")
print(f"  False Positives: {cm_rf[0,1]:,}")
print(f"  False Negatives: {cm_rf[1,0]:,}")
print(f"  True Positives:  {cm_rf[1,1]:,}")

### Compare all models

In [ ]:
# =========================
# % IMPROVEMENTS
# =========================
rf_vs_base = ((rf_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0
logit_vs_base = ((logit_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0
xgb_vs_base = ((xgb_f1 - baseline_f1) / baseline_f1) * 100 if baseline_f1 > 0 else 0

rf_vs_logit = ((rf_f1 - logit_f1) / logit_f1) * 100 if logit_f1 > 0 else 0
xgb_vs_logit = ((xgb_f1 - logit_f1) / logit_f1) * 100 if logit_f1 > 0 else 0
xgb_vs_rf = ((xgb_f1 - rf_f1) / rf_f1) * 100 if rf_f1 > 0 else 0

print("\n" + "="*70)
print("COMPARISON (BASELINE vs LOGISTIC vs RANDOM FOREST vs XGBOOST)")
print("="*70)

print(f"Baseline F1 (Route-Based):     {baseline_f1:.4f}")
print(f"Logistic Regression F1:        {logit_f1:.4f}   ({logit_vs_base:+.1f}% vs baseline)")
print(f"Random Forest F1:              {rf_f1:.4f}   ({rf_vs_base:+.1f}% vs baseline)")
print(f"XGBoost F1:                    {xgb_f1:.4f}   ({xgb_vs_base:+.1f}% vs baseline)")
print("-"*70)
print(f"Random Forest vs Logistic (F1): {rf_vs_logit:+.1f}%")
print(f"XGBoost vs Logistic (F1):       {xgb_vs_logit:+.1f}%")
print(f"XGBoost vs Random Forest (F1):  {xgb_vs_rf:+.1f}%")

print("\nDetailed Metrics (Test Set):")
print(f"{'Model':<22} {'Acc':>8} {'Prec':>8} {'Recall':>8} {'F1':>8}")
print(f"{'-'*22} {'-'*8:>8} {'-'*8:>8} {'-'*8:>8} {'-'*8:>8}")
print(f"{'Baseline (Route)':<22} {baseline_accuracy:>8.4f} {baseline_precision:>8.4f} {baseline_recall:>8.4f} {baseline_f1:>8.4f}")
print(f"{'Logistic Regression':<22} {logit_accuracy:>8.4f} {logit_precision:>8.4f} {logit_recall:>8.4f} {logit_f1:>8.4f}")
print(f"{'Random Forest':<22} {rf_accuracy:>8.4f} {rf_precision:>8.4f} {rf_recall:>8.4f} {rf_f1:>8.4f}")
print(f"{'XGBoost':<22} {xgb_accuracy:>8.4f} {xgb_precision:>8.4f} {xgb_recall:>8.4f} {xgb_f1:>8.4f}")

# =========================
# SIMPLE WINNER SUMMARY
# =========================
print("\n" + "="*70)
print("RESULT SUMMARY")
print("="*70)

best_model = max(
    [
        ("Baseline (Route)", baseline_f1),
        ("Logistic Regression", logit_f1),
        ("Random Forest", rf_f1),
        ("XGBoost", xgb_f1),
    ],
    key=lambda x: x[1]
)[0]

if best_model == "XGBoost" and xgb_f1 > baseline_f1 and xgb_f1 > rf_f1 and xgb_f1 > logit_f1:
    print(" BEST: XGBoost is the top performer and beats Logistic, Random Forest, and Baseline.")
elif best_model == "Random Forest" and rf_f1 > baseline_f1 and rf_f1 > logit_f1 and rf_f1 > xgb_f1:
    print(" BEST: Random Forest is the top performer and beats Logistic, XGBoost, and Baseline.")
elif best_model == "Logistic Regression" and logit_f1 > baseline_f1 and logit_f1 > rf_f1 and logit_f1 > xgb_f1:
    print(" BEST: Logistic Regression is the top performer and beats Random Forest, XGBoost, and Baseline.")
elif best_model == "Baseline (Route)":
    print("  Baseline is still the strongest by F1. Consider threshold tuning or more features.")
elif xgb_f1 > baseline_f1:
    print(" XGBoost beats the Baseline, but does not beat all other ML models.")
elif rf_f1 > baseline_f1:
    print(" Random Forest beats the Baseline, but does not beat all other ML models.")
elif logit_f1 > baseline_f1:
    print(" Logistic beats the Baseline, but does not beat all other ML models.")
else:
    print(" Neither Logistic, Random Forest, nor XGBoost beats the Baseline.")


### Comparision to the best model 

In [ ]:
# =========================
# COMPARE LOGISTIC vs RANDOM FOREST
# =========================
model_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [logit_accuracy, rf_accuracy],
    'Precision': [logit_precision, rf_precision],
    'Recall': [logit_recall, rf_recall],
    'F1 Score': [logit_f1, rf_f1]
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
colors = ['#e74c3c', "#3498db"]  # logistic, random forest

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(
        model_comparison['Model'],
        model_comparison[metric],
        color=colors,
        edgecolor='black',
        alpha=0.7
    )
    ax.set_ylabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='x', rotation=15)
    
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.,
            height,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold'
        )

plt.suptitle('Final Model Performance Comparison\n(Logistic Regression vs Random Forest)',
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n" + model_comparison.to_string(index=False))


In [ ]:
# =========================
# COMPARE ALL MODELS (BASELINE vs LOGISTIC vs RANDOM FOREST vs XGBOOST)
# =========================
model_comparison = pd.DataFrame({
    'Model': ['Baseline (Route)', 'Logistic Regression', 'Random Forest', 'XGBoost'],
    'Accuracy': [baseline_accuracy, logit_accuracy, rf_accuracy, xgb_accuracy],
    'Precision': [baseline_precision, logit_precision, rf_precision, xgb_precision],
    'Recall': [baseline_recall, logit_recall, rf_recall, xgb_recall],
    'F1 Score': [baseline_f1, logit_f1, rf_f1, xgb_f1]
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

# 4 models -> 4 colors (same style as your template)
colors = ['#95a5a6', '#e74c3c', '#3498db', '#2ecc71']  # baseline, logistic, rf, xgb

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    bars = ax.bar(
        model_comparison['Model'],
        model_comparison[metric],
        color=colors,
        edgecolor='black',
        alpha=0.7
    )
    ax.set_ylabel(metric, fontsize=12)
    ax.set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3, axis='y')
    ax.tick_params(axis='x', rotation=15)

    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width() / 2.,
            height,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontsize=10,
            fontweight='bold'
        )

plt.suptitle(
    'Final Model Performance Comparison\n(Baseline vs Logistic Regression vs Random Forest vs XGBoost)',
    fontsize=16,
    fontweight='bold'
)
plt.tight_layout()
plt.show()

print("\n" + model_comparison.to_string(index=False))


---
# 11 Tunning threshold for Logistic model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    precision_recall_curve,
)


# 1) Fit logistic model (on SCALED features)
logit_model = LogisticRegression(
    class_weight="balanced",
    C=0.1,
    max_iter=1000,
    solver="lbfgs",
    random_state=42,
    n_jobs=-1
)

logit_model.fit(X_train_scaled, y_train)
print(" Logistic model fitted (Option A: manual scaling).")

# 2) Get predicted probabilities (class=1)
y_proba_train = logit_model.predict_proba(X_train_scaled)[:, 1]
y_proba_test  = logit_model.predict_proba(X_test_scaled)[:, 1]

# 3) Manually tune threshold for higher recall
threshold_grid = np.array([0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.20, 0.15, 0.10])

rows = []
for t in threshold_grid:
    y_pred_t = (y_proba_test >= t).astype(int)
    rows.append({
        "threshold": t,
        "precision": precision_score(y_test, y_pred_t, zero_division=0),
        "recall": recall_score(y_test, y_pred_t, zero_division=0),
        "f1": f1_score(y_test, y_pred_t, zero_division=0),
        "pred_pos_rate": y_pred_t.mean()
    })

thresh_table = pd.DataFrame(rows).sort_values("threshold", ascending=False)
display(thresh_table)

# 3b) Pick a threshold that meets a recall target
RECALL_TARGET = 0.80  # <-- change this as needed (e.g., 0.85)

prec, rec, thr = precision_recall_curve(y_test, y_proba_test)
prec_aligned = prec[:-1]
rec_aligned  = rec[:-1]

eligible = np.where(rec_aligned >= RECALL_TARGET)[0]

if len(eligible) == 0:
    best_threshold = 0.50
    print(f" No threshold reached recall >= {RECALL_TARGET:.2f}. Defaulting to 0.50")
else:
    # choose the HIGHEST threshold that still meets recall target (reduces false positives)
    best_idx = eligible[np.argmax(thr[eligible])]
    best_threshold = float(thr[best_idx])
    print(f" Chosen threshold for recall >= {RECALL_TARGET:.2f}: {best_threshold:.4f}")

# 4) Evaluate at the chosen threshold
y_pred_test_thresh = (y_proba_test >= best_threshold).astype(int)

print("\n=== Confusion Matrix (Test) ===")
print(confusion_matrix(y_test, y_pred_test_thresh))

print("\n=== Classification Report (Test) ===")
print(classification_report(y_test, y_pred_test_thresh, digits=4))

print(f"Threshold used: {best_threshold:.4f}")
print(f"Test Precision: {precision_score(y_test, y_pred_test_thresh, zero_division=0):.4f}")
print(f"Test Recall:    {recall_score(y_test, y_pred_test_thresh, zero_division=0):.4f}")
print(f"Test F1:        {f1_score(y_test, y_pred_test_thresh, zero_division=0):.4f}")

# 5) Save the threshold for later use/deployment
BEST_THRESHOLD = best_threshold


In [ ]:
import shap
import matplotlib.pyplot as plt

# --- Make sure feature names align with your scaled arrays ---
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_cols)
X_test_scaled_df  = pd.DataFrame(X_test_scaled,  columns=feature_cols)

# --- (Recommended) sample for speed (SHAP can be heavy on huge test sets) ---
bg_size = min(2000, len(X_train_scaled_df))   # background for expected value
eval_size = min(5000, len(X_test_scaled_df))  # rows to explain/plot

X_bg = X_train_scaled_df.sample(bg_size, random_state=42)
X_eval = X_test_scaled_df.sample(eval_size, random_state=42)

# --- Linear SHAP explainer for logistic regression ---
# For binary classification, SHAP values are usually in "log-odds" space for linear models.
explainer = shap.LinearExplainer(logit_model, X_bg, feature_perturbation="interventional")

shap_values = explainer.shap_values(X_eval)  # shape: (n_eval, n_features)

# --- Global feature importance: mean absolute SHAP ---
importance = np.abs(shap_values).mean(axis=0)
shap_importance_df = (
    pd.DataFrame({"feature": feature_cols, "mean_abs_shap": importance})
      .sort_values("mean_abs_shap", ascending=False)
)

display(shap_importance_df.head(15))

In [ ]:
# Bar plot: global importance (mean(|SHAP|))
plt.figure(figsize=(10, 6))
top = shap_importance_df.head(10).iloc[::-1]  # reverse for horizontal bar
plt.barh(top["feature"], top["mean_abs_shap"])
plt.xlabel("Mean(|SHAP value|) across samples")
plt.title("Top 10 Feature Importance (SHAP)")
plt.tight_layout()
plt.show()

# Beeswarm summary plot (shows direction + magnitude across samples)
shap.summary_plot(shap_values, X_eval, feature_names=feature_cols, show=True)


---
<a id="question-11"></a>
# Key Summary

**Summary:**
- Logistic regression achieved F1 = 0.2922 (vs baseline 0.0871)
- **235.4% improvement** over baseline!
- Reasonable precision (30%) - better than baseline
- Most important features: stop_delay_rate, route_delay_rate, stop_sequence
- Temporal features had minimal impact (likely due to single snapshot)

---
<a id="summary"></a>
# Summary and Conclusions

## Overall Results

| Question | Task | Key Metric | Result |
|----------|------|------------|--------|
| **6** | Exploratory Analysis | Class Imbalance | 11.4:1 (8.05% delays) |
| **7** | Baseline Model | Best F1 Score | 0.0871 (Route-Based) |
| **8** | Logistic Regression | F1 Score | 0.2922 (+235.4% improvement) |
| **9** | XgBoost Regression | F1 Score | 0.2344 (+169.0% improvement) |
| **10** | Random Forest Regression | F1 Score | 0.2912 (+234.3% improvement) |

## Key Insights

1. **Data Characteristics:**
   - Most buses run on time (median delay: -0.18 min)
   - Major delays (≥10 min) are relatively rare (8.05%)
   - Significant class imbalance requires special handling

2. **Baseline Performance:**
   - Majority class baseline is useless despite 93% accuracy
   - Route-based baseline shows route information is predictive
   - Time-based features alone are not strong predictors

3. **Logistic Regression Success:**
   - Achieved 90% accuracy - catches almost all delays
   - 0.2922 F1 score - the best F1 so far,+235.4% improvement from the baseline
   - Stop and route historical patterns are strongest predictors
   - Successfully handles class imbalance with balanced weighting

## Limitations

- **Single data snapshot:** Limited temporal diversity
- **Modest precision:** Many false alarms (but acceptable for warnings)
- **Temporal features underutilized:** Need data across different times/days

## Future Work

1. **Data Collection:** Gather data over 1-2 weeks for better temporal patterns
2. **Feature Engineering:** Add bus bunching, weather, special events
3. **Model Improvements:** Try tree-based models, tune thresholds
4. **Deployment:** Build real-time prediction API for commuters

## Conclusion

Despite limited data, we successfully:
- ✅ Explored and understood the dataset
- ✅ Established meaningful baselines
- ✅ Built a logistic regression model that significantly outperforms baselines
- ✅ Identified key delay predictors (stop/route history)

The 10-minute delay threshold provides a good balance between actionability and statistical feasibility. With additional data collection, we expect model performance to improve further, making this a viable tool for transit delay prediction.